In [42]:
import tensorflow as tf
import numpy as np

# Download MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
test_features = mnist.test.images

# Parameters 
learning_rate = 0.00001
epochs = 2
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [43]:
# Weights and biases
weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [44]:
# Convolution
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x,W,strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [45]:
# Max Pooling
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [52]:
def conv_net(x, weights, biases, dropout):
    # Layer1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer1 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [54]:
# RUN !!
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

print(logits)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #print(batch_x.shape)
            #print(batch_y.shape)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})
            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
    
    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Tensor("Add_39:0", shape=(?, 10), dtype=float32)
Epoch  1, Batch   1 -Loss: 45516.9297 Validation Accuracy: 0.113281
Epoch  1, Batch   2 -Loss: 38186.4141 Validation Accuracy: 0.128906
Epoch  1, Batch   3 -Loss: 33114.6016 Validation Accuracy: 0.144531
Epoch  1, Batch   4 -Loss: 30895.6484 Validation Accuracy: 0.164062
Epoch  1, Batch   5 -Loss: 29632.1934 Validation Accuracy: 0.164062
Epoch  1, Batch   6 -Loss: 28975.6113 Validation Accuracy: 0.164062
Epoch  1, Batch   7 -Loss: 28973.3809 Validation Accuracy: 0.175781
Epoch  1, Batch   8 -Loss: 27520.9141 Validation Accuracy: 0.183594
Epoch  1, Batch   9 -Loss: 24072.1309 Validation Accuracy: 0.199219
Epoch  1, Batch  10 -Loss: 23374.6172 Validation Accuracy: 0.199219
Epoch  1, Batch  11 -Loss: 23579.2500 Validation Accuracy: 0.203125
Epoch  1, Batch  12 -Loss: 21323.8008 Validation Accuracy: 0.222656
Epoch  1, Batch  13 -Loss: 21072.3438 Validation Accuracy: 0.234375
Epoch  1, Batch  14 -Loss: 18253.6211 Validation Accuracy: 0.246094

Epoch  1, Batch 121 -Loss:  4344.5220 Validation Accuracy: 0.644531
Epoch  1, Batch 122 -Loss:  3244.8406 Validation Accuracy: 0.644531
Epoch  1, Batch 123 -Loss:  3706.0674 Validation Accuracy: 0.648438
Epoch  1, Batch 124 -Loss:  3484.4534 Validation Accuracy: 0.656250
Epoch  1, Batch 125 -Loss:  3317.6733 Validation Accuracy: 0.652344
Epoch  1, Batch 126 -Loss:  4069.8577 Validation Accuracy: 0.648438
Epoch  1, Batch 127 -Loss:  3835.2942 Validation Accuracy: 0.644531
Epoch  1, Batch 128 -Loss:  4171.1128 Validation Accuracy: 0.648438
Epoch  1, Batch 129 -Loss:  5201.5166 Validation Accuracy: 0.648438
Epoch  1, Batch 130 -Loss:  3102.5400 Validation Accuracy: 0.652344
Epoch  1, Batch 131 -Loss:  2789.7268 Validation Accuracy: 0.648438
Epoch  1, Batch 132 -Loss:  4104.5986 Validation Accuracy: 0.660156
Epoch  1, Batch 133 -Loss:  2944.0317 Validation Accuracy: 0.671875
Epoch  1, Batch 134 -Loss:  4665.5664 Validation Accuracy: 0.664062
Epoch  1, Batch 135 -Loss:  3131.9917 Validation

Epoch  1, Batch 242 -Loss:  2276.5376 Validation Accuracy: 0.722656
Epoch  1, Batch 243 -Loss:  2283.9316 Validation Accuracy: 0.710938
Epoch  1, Batch 244 -Loss:  1928.7186 Validation Accuracy: 0.707031
Epoch  1, Batch 245 -Loss:  1857.9124 Validation Accuracy: 0.722656
Epoch  1, Batch 246 -Loss:  1699.0040 Validation Accuracy: 0.722656
Epoch  1, Batch 247 -Loss:  2192.2903 Validation Accuracy: 0.722656
Epoch  1, Batch 248 -Loss:  2222.0332 Validation Accuracy: 0.722656
Epoch  1, Batch 249 -Loss:  1599.2266 Validation Accuracy: 0.722656
Epoch  1, Batch 250 -Loss:  2119.8149 Validation Accuracy: 0.726562
Epoch  1, Batch 251 -Loss:  2643.4897 Validation Accuracy: 0.722656
Epoch  1, Batch 252 -Loss:  1995.9124 Validation Accuracy: 0.722656
Epoch  1, Batch 253 -Loss:  2217.0857 Validation Accuracy: 0.726562
Epoch  1, Batch 254 -Loss:  1995.8914 Validation Accuracy: 0.718750
Epoch  1, Batch 255 -Loss:  2371.9854 Validation Accuracy: 0.722656
Epoch  1, Batch 256 -Loss:  2097.2957 Validation

Epoch  1, Batch 363 -Loss:  1273.3900 Validation Accuracy: 0.746094
Epoch  1, Batch 364 -Loss:  2085.9443 Validation Accuracy: 0.750000
Epoch  1, Batch 365 -Loss:  1710.3616 Validation Accuracy: 0.750000
Epoch  1, Batch 366 -Loss:  1373.2720 Validation Accuracy: 0.742188
Epoch  1, Batch 367 -Loss:  1850.0103 Validation Accuracy: 0.742188
Epoch  1, Batch 368 -Loss:  2170.8918 Validation Accuracy: 0.742188
Epoch  1, Batch 369 -Loss:  1967.6802 Validation Accuracy: 0.738281
Epoch  1, Batch 370 -Loss:  1312.8588 Validation Accuracy: 0.746094
Epoch  1, Batch 371 -Loss:  1986.2588 Validation Accuracy: 0.746094
Epoch  1, Batch 372 -Loss:  1386.0186 Validation Accuracy: 0.750000
Epoch  1, Batch 373 -Loss:  1470.3024 Validation Accuracy: 0.746094
Epoch  1, Batch 374 -Loss:  1472.2982 Validation Accuracy: 0.742188
Epoch  1, Batch 375 -Loss:  2104.2336 Validation Accuracy: 0.742188
Epoch  1, Batch 376 -Loss:  1113.4917 Validation Accuracy: 0.738281
Epoch  1, Batch 377 -Loss:  1744.4510 Validation

Epoch  2, Batch  55 -Loss:  1302.6257 Validation Accuracy: 0.750000
Epoch  2, Batch  56 -Loss:  1101.9700 Validation Accuracy: 0.753906
Epoch  2, Batch  57 -Loss:  1670.9684 Validation Accuracy: 0.750000
Epoch  2, Batch  58 -Loss:  1441.5505 Validation Accuracy: 0.746094
Epoch  2, Batch  59 -Loss:  1481.4818 Validation Accuracy: 0.757812
Epoch  2, Batch  60 -Loss:   806.1849 Validation Accuracy: 0.761719
Epoch  2, Batch  61 -Loss:  1390.8427 Validation Accuracy: 0.753906
Epoch  2, Batch  62 -Loss:  1227.1948 Validation Accuracy: 0.757812
Epoch  2, Batch  63 -Loss:  1260.9777 Validation Accuracy: 0.761719
Epoch  2, Batch  64 -Loss:  1396.9988 Validation Accuracy: 0.761719
Epoch  2, Batch  65 -Loss:  1040.2450 Validation Accuracy: 0.765625
Epoch  2, Batch  66 -Loss:  1189.9363 Validation Accuracy: 0.761719
Epoch  2, Batch  67 -Loss:  1606.5288 Validation Accuracy: 0.761719
Epoch  2, Batch  68 -Loss:  1225.7462 Validation Accuracy: 0.761719
Epoch  2, Batch  69 -Loss:  1348.1609 Validation

...

Epoch  9, Batch 319 -Loss:   326.0138 Validation Accuracy: 0.832031
Epoch  9, Batch 320 -Loss:   250.5722 Validation Accuracy: 0.832031
Epoch  9, Batch 321 -Loss:   266.9249 Validation Accuracy: 0.835938
Epoch  9, Batch 322 -Loss:   229.0598 Validation Accuracy: 0.832031
Epoch  9, Batch 323 -Loss:   382.5893 Validation Accuracy: 0.835938
Epoch  9, Batch 324 -Loss:   365.3996 Validation Accuracy: 0.847656
Epoch  9, Batch 325 -Loss:   212.1403 Validation Accuracy: 0.839844
Epoch  9, Batch 326 -Loss:   308.0683 Validation Accuracy: 0.843750
Epoch  9, Batch 327 -Loss:   233.7996 Validation Accuracy: 0.851562
Epoch  9, Batch 328 -Loss:   234.9283 Validation Accuracy: 0.843750
Epoch  9, Batch 329 -Loss:   249.1530 Validation Accuracy: 0.851562
Epoch  9, Batch 330 -Loss:   153.0142 Validation Accuracy: 0.847656
Epoch  9, Batch 331 -Loss:   198.5462 Validation Accuracy: 0.851562
Epoch  9, Batch 332 -Loss:   261.1813 Validation Accuracy: 0.851562
Epoch  9, Batch 333 -Loss:   249.6097 Validation

Epoch 10, Batch  11 -Loss:   174.8224 Validation Accuracy: 0.839844
Epoch 10, Batch  12 -Loss:   340.9877 Validation Accuracy: 0.832031
Epoch 10, Batch  13 -Loss:   374.8907 Validation Accuracy: 0.828125
Epoch 10, Batch  14 -Loss:   286.5266 Validation Accuracy: 0.828125
Epoch 10, Batch  15 -Loss:   362.0711 Validation Accuracy: 0.839844
Epoch 10, Batch  16 -Loss:   354.6614 Validation Accuracy: 0.835938
Epoch 10, Batch  17 -Loss:   334.1500 Validation Accuracy: 0.839844
Epoch 10, Batch  18 -Loss:   125.3809 Validation Accuracy: 0.835938
Epoch 10, Batch  19 -Loss:   255.7291 Validation Accuracy: 0.839844
Epoch 10, Batch  20 -Loss:   369.7460 Validation Accuracy: 0.843750
Epoch 10, Batch  21 -Loss:   195.2486 Validation Accuracy: 0.832031
Epoch 10, Batch  22 -Loss:   299.4856 Validation Accuracy: 0.835938
Epoch 10, Batch  23 -Loss:   116.1253 Validation Accuracy: 0.832031
Epoch 10, Batch  24 -Loss:   374.9401 Validation Accuracy: 0.828125
Epoch 10, Batch  25 -Loss:   181.4341 Validation

Epoch 10, Batch 132 -Loss:   280.8239 Validation Accuracy: 0.839844
Epoch 10, Batch 133 -Loss:   319.4796 Validation Accuracy: 0.843750
Epoch 10, Batch 134 -Loss:   343.5172 Validation Accuracy: 0.839844
Epoch 10, Batch 135 -Loss:   281.3634 Validation Accuracy: 0.835938
Epoch 10, Batch 136 -Loss:   225.8115 Validation Accuracy: 0.835938
Epoch 10, Batch 137 -Loss:   272.8495 Validation Accuracy: 0.832031
Epoch 10, Batch 138 -Loss:   328.8250 Validation Accuracy: 0.835938
Epoch 10, Batch 139 -Loss:   218.9085 Validation Accuracy: 0.835938
Epoch 10, Batch 140 -Loss:   160.2142 Validation Accuracy: 0.832031
Epoch 10, Batch 141 -Loss:   323.3830 Validation Accuracy: 0.835938
Epoch 10, Batch 142 -Loss:   248.8303 Validation Accuracy: 0.839844
Epoch 10, Batch 143 -Loss:   164.0625 Validation Accuracy: 0.839844
Epoch 10, Batch 144 -Loss:   246.1127 Validation Accuracy: 0.835938
Epoch 10, Batch 145 -Loss:   234.7271 Validation Accuracy: 0.835938
Epoch 10, Batch 146 -Loss:   252.0412 Validation

Epoch 10, Batch 253 -Loss:   298.0368 Validation Accuracy: 0.828125
Epoch 10, Batch 254 -Loss:   291.9987 Validation Accuracy: 0.835938
Epoch 10, Batch 255 -Loss:   205.1136 Validation Accuracy: 0.828125
Epoch 10, Batch 256 -Loss:   270.6165 Validation Accuracy: 0.832031
Epoch 10, Batch 257 -Loss:   340.4895 Validation Accuracy: 0.832031
Epoch 10, Batch 258 -Loss:   159.9849 Validation Accuracy: 0.828125
Epoch 10, Batch 259 -Loss:   192.5155 Validation Accuracy: 0.828125
Epoch 10, Batch 260 -Loss:   267.4273 Validation Accuracy: 0.828125
Epoch 10, Batch 261 -Loss:   275.2579 Validation Accuracy: 0.832031
Epoch 10, Batch 262 -Loss:   269.7844 Validation Accuracy: 0.835938
Epoch 10, Batch 263 -Loss:   137.3786 Validation Accuracy: 0.832031
Epoch 10, Batch 264 -Loss:   232.9483 Validation Accuracy: 0.828125
Epoch 10, Batch 265 -Loss:   200.0570 Validation Accuracy: 0.828125
Epoch 10, Batch 266 -Loss:   288.0319 Validation Accuracy: 0.832031
Epoch 10, Batch 267 -Loss:   226.0365 Validation

Epoch 10, Batch 374 -Loss:   214.2360 Validation Accuracy: 0.828125
Epoch 10, Batch 375 -Loss:   153.4659 Validation Accuracy: 0.832031
Epoch 10, Batch 376 -Loss:   173.6964 Validation Accuracy: 0.835938
Epoch 10, Batch 377 -Loss:   221.1217 Validation Accuracy: 0.835938
Epoch 10, Batch 378 -Loss:   181.6473 Validation Accuracy: 0.835938
Epoch 10, Batch 379 -Loss:   210.9439 Validation Accuracy: 0.828125
Epoch 10, Batch 380 -Loss:   245.0377 Validation Accuracy: 0.832031
Epoch 10, Batch 381 -Loss:   263.1433 Validation Accuracy: 0.832031
Epoch 10, Batch 382 -Loss:   369.8906 Validation Accuracy: 0.828125
Epoch 10, Batch 383 -Loss:   111.2041 Validation Accuracy: 0.832031
Epoch 10, Batch 384 -Loss:   320.4876 Validation Accuracy: 0.832031
Epoch 10, Batch 385 -Loss:   302.9428 Validation Accuracy: 0.828125
Epoch 10, Batch 386 -Loss:   150.9409 Validation Accuracy: 0.832031
Epoch 10, Batch 387 -Loss:   221.3393 Validation Accuracy: 0.832031
Epoch 10, Batch 388 -Loss:   262.6981 Validation